In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/utils")
from utils import SimpleGraphVoltDatasetLoader, read_and_prepare_data

In [2]:
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

In [3]:
trafo_id = "T1330"
loader = SimpleGraphVoltDatasetLoader(trafo_id)

   node_id  power_production  power_consumption  PMO  TR  junction  \
0        0         -0.669711          -0.211164    0   1         0   

             date_time   voltage  temperature_2m  snow_depth  ...  \
0  2021-06-01 00:00:00  0.871501       -0.087913   -0.373525  ...   

   direct_radiation  diffuse_radiation  direct_normal_irradiance  \
0         -0.563245          -0.758562                 -0.682145   

   active_power  reactive_power      year     month       day      hour  \
0      7.244788        5.311676 -1.301898 -0.152567 -1.673503 -1.661325   

     minute  
0 -1.341641  

[1 rows x 23 columns]


In [4]:
loader_data = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=4)

(2, 114)
(114, 5)
(114,)
(70065, 113, 21, 12)
(70065, 113, 4)


In [5]:
from torch_geometric_temporal.signal import temporal_signal_split

In [6]:
train_dataset, test_dataset = temporal_signal_split(loader_data, train_ratio=0.2)

In [7]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN

class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods):
        super(TemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        out_channels = 32
        self.tgnn = A3TGCN(in_channels=node_features, 
                           out_channels=out_channels, 
                           periods=periods)
        # Equals single-shot prediction
        self.linear = torch.nn.Linear(out_channels, periods)

    def forward(self, x, edge_index):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        """
        h = self.tgnn(x, edge_index)
        h = F.relu(h)
        h = self.linear(h)
        return h

TemporalGNN(node_features=16, periods=4)

TemporalGNN(
  (tgnn): A3TGCN(
    (_base_tgcn): TGCN(
      (conv_z): GCNConv(16, 32)
      (linear_z): Linear(in_features=64, out_features=32, bias=True)
      (conv_r): GCNConv(16, 32)
      (linear_r): Linear(in_features=64, out_features=32, bias=True)
      (conv_h): GCNConv(16, 32)
      (linear_h): Linear(in_features=64, out_features=32, bias=True)
    )
  )
  (linear): Linear(in_features=32, out_features=4, bias=True)
)

In [9]:
# GPU support
device = torch.device('cuda') # cuda
subset = 2000

# Create model and optimizers
model = TemporalGNN(node_features=21, periods=4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.train()

epochs_loss = []

print("Running training...")
for epoch in range(30): 
    loss = 0
    step = 0
    print(f"--------- Epoch {epoch} ---------")
    for snapshot in train_dataset:
        #print(f"--- Step {step} ---")
        snapshot = snapshot.to(device)
        # Get model predictions
        y_hat = model(snapshot.x, snapshot.edge_index)
        # Mean squared error
        intermedaiate_loss = torch.mean((y_hat-snapshot.y)**2) 
        loss = loss + intermedaiate_loss
        step += 1
        if step%1000==0:
          print(f"Intermediate loss at step {step}: {intermedaiate_loss.item()}")
        if step > subset:
          break

    loss = loss / (step + 1)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("Epoch {} train MSE: {:.4f}".format(epoch, loss.item()))
    epochs_loss.append(loss)

Running training...
--------- Epoch 0 ---------
Intermediate loss at step 1000: 1.0177550315856934
Intermediate loss at step 2000: 1.0104764699935913
Epoch 0 train MSE: 1.0108
--------- Epoch 1 ---------
Intermediate loss at step 1000: 0.9886978268623352
Intermediate loss at step 2000: 0.9905419945716858
Epoch 1 train MSE: 0.9871
--------- Epoch 2 ---------
Intermediate loss at step 1000: 0.9664047956466675
Intermediate loss at step 2000: 0.9696627259254456
Epoch 2 train MSE: 0.9664
--------- Epoch 3 ---------
Intermediate loss at step 1000: 0.9379709362983704
Intermediate loss at step 2000: 0.9398724436759949
Epoch 3 train MSE: 0.9389
--------- Epoch 4 ---------
Intermediate loss at step 1000: 0.9015889167785645
Intermediate loss at step 2000: 0.9045805931091309
Epoch 4 train MSE: 0.9040
--------- Epoch 5 ---------
Intermediate loss at step 1000: 0.8642348647117615
Intermediate loss at step 2000: 0.8645107746124268
Epoch 5 train MSE: 0.8586
--------- Epoch 6 ---------
Intermediate los